Initial questions:

- How can I load this data?
- What other pre-processing does it require?
    - Background noise estimation
- What networks do I need to build?
    - Their code literally gives you all the layers and hyperparameters
- How do I train them?
    - DNN
        - Chunks
        - Batches
        - Multiple epochs
        - Binary cross-entropy objective function
        - Stochastic Gradient Descent optimization
    - DAE
        - Multiple-frame MBK to predict center (context)
        - Batches
        - Multiple epochs
        - Mean squared error objective function
        - Stochastic Gradient Descent optimization
- How do I evaluate them?
    - Equal error rate, average across 5 folds
    - Precision
    - Recall
    - F1

Notes:
- Training and network hyperparameters are given
- Will use the given training and development folds, and 16kHz rather than 48kHz
- Will have to reconstruct their input pre-processing manually based on the inputs to the different networks, because it is "done offstage" in this repo, something something MFCCs and MBKs and background noise (check out their equation, and see if the second paper does something similar), fortunately they give you a lot of the different specs but you may just need to note in your report that you're kinda guessing, don't have to match their performance either



In [2]:
] activate .

  Activating environment at `~/Desktop/csce-5380-project/Project.toml`


In [27]:
using Pkg
using WAV
using CSV
using DelimitedFiles
using DataFrames
using GoogleDrive
using DataSets
using Flux
using Plots
using StatsPlots
using OhMyREPL

In [19]:
project = DataSets.load_project(Dict(
    "data_config_version"=>0,
    "datasets"=>[Dict(
        "name"=>"chime_home",
        "uuid"=>"73b60068-bf34-11eb-17e9-5fa4ccb60cd2", # UUID generated externally, no need to change
        "description"=>"4-second audio chunks at different sample rates with labels and other metadata",
        "storage"=>Dict(
            "driver"=>"FileSystem",
            "type"=>"BlobTree",
            "path"=>joinpath(homedir(), ".julia", "datadeps", "chime_home")
        )
    )]
))

DataSets.DataProject:
  📁 chime_home => 73b60068-bf34-11eb-17e9-5fa4ccb60cd2

In [20]:
chime_home = open(BlobTree, dataset(project, "chime_home"))

📂 Tree  @ /home/jhigginbotham64/.julia/datadeps/chime_home
 📄 CHANGES
 📄 LICENSE
 📄 README
 📄 VERSION
 📁 chunks
 📄 development_chunks_raw.csv
 📄 development_chunks_refined.csv
 📄 development_chunks_refined_crossval_dcase2016.csv
 📄 evaluation_chunks_raw.csv
 📄 evaluation_chunks_refined.csv

In [21]:
# this function can nicely display everything
# in chime_home except the chunks directory
function print_data_file(s)
    open(String, chime_home[s]) do data
        print(data)
    end
end

print_data_file("README") # description of data set useful for report, might even just include as output

Summary
-------------------
The audio recordings and annotations included in this archive form the CHiME-Home dataset. To cite this dataset, please use the reference provided at the end of this document. Please refer to the same publication for a more detailed description of the dataset.

The CHiME-Home dataset is a collection of annotated domestic environment audio recordings. The audio recordings were originally made for the CHiME project (cf. Christensen et al., 2010; Barker et al., 2013). In the CHiME-Home dataset, 4-second audio chunks are each associated with multiple labels, based on a set of 7 labels associated with sound sources in the acoustic environment. For a total of 6137 4-second audio chunks, there are 3 sets of multi-label annotations. A subset of the 6137 chunks (1946 chunks) is further specified, for which annotators' label assignments agree strongly (cf. Foster et al., 2015). For clearer distinction, please refer to the 2762-chunk dataset as CHiME-Home-refine; corre

In [29]:
# read all chunk grouping CSV files into 
# DataFrames with appropriate headers
chunk_headers=["id", "name"]
dev_chunks_raw = CSV.read(
    IOBuffer(open(String, chime_home["development_chunks_raw.csv"])), 
    DataFrame; header=chunk_headers);
dev_chunks_refined = CSV.read(
    IOBuffer(open(String, chime_home["development_chunks_refined.csv"])), 
    DataFrame; header=chunk_headers);
eval_chunks_raw = CSV.read(
    IOBuffer(open(String, chime_home["evaluation_chunks_raw.csv"])), 
    DataFrame; header=chunk_headers);
eval_chunks_refined = CSV.read(
    IOBuffer(open(String, chime_home["evaluation_chunks_refined.csv"])), 
    DataFrame; header=chunk_headers);
dev_chunks_refined_folds = CSV.read(
    IOBuffer(open(String, chime_home["development_chunks_refined_crossval_dcase2016.csv"])), 
    DataFrame; header=vcat(chunk_headers, ["fold"]));

In [59]:
chunks = chime_home["chunks"]

📂 Tree chunks @ /home/ubuntu/csce-5380-project/chime_home
 📄 CR_lounge_200110_1601.s0_chunk0.16kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk0.48kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk0.csv
 📄 CR_lounge_200110_1601.s0_chunk1.16kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk1.48kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk1.csv
 📄 CR_lounge_200110_1601.s0_chunk10.16kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk10.48kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk10.csv
 📄 CR_lounge_200110_1601.s0_chunk11.16kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk11.48kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk11.csv
 📄 CR_lounge_200110_1601.s0_chunk12.16kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk12.48kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk12.csv
 📄 CR_lounge_200110_1601.s0_chunk13.16kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk13.48kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk13.csv
 📄 CR_lounge_200110_1601.s0_chunk14.16kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk14.48kHz.wav
 📄 CR_lounge_200110_1601.s0_chunk14.csv
 📄 CR_lounge_200110_1601

In [69]:
#=
    for my next trick, i will aggregate the entire data set into
    one data frame, with the CSV information for each chunk together
    with the 16kHz and 48kHz sound data.

    ...actually, what i want to do is make a function that retrieves
    all that data given a chunk name, which means not having to search
    the chunks directory, and also means that i can construct data sets
    from the given CSV files on-demand and release them when i'm finished
    with them.

    unsure how to format individual elements for usage with DataFrames,
    except perhaps as 1-element DataFrames. that might work.
=#

function chunk(name::AbstractString)
    return false;
end

chunk (generic function with 1 method)